In [54]:
import tensorflow as tf
import os
import matplotlib.pyplot as plt
from PIL import Image

In [55]:
base_dir = 'D:/dataset/programmers/'
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')
train_dir, test_dir

('D:/dataset/programmers/train', 'D:/dataset/programmers/test')

In [56]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [57]:
IMG_SIZE = (224,224)
generator = ImageDataGenerator(rotation_range=40,
                          width_shift_range=0.2,
                          height_shift_range=0.2,
                          shear_range=0.2,
                          zoom_range=0.2,
                          horizontal_flip=True,
                          vertical_flip=True,
                          fill_mode = 'reflect',
                          validation_split = 0.3,
                          rescale=1/255.)

test_generator = ImageDataGenerator(rescale=1/255.)
# print(help(ImageDataGenerator))

In [58]:
train_gen = generator.flow_from_directory(
            train_dir,
            target_size=IMG_SIZE,
            class_mode='categorical',
            shuffle=True,
            subset='training',)
val_gen = generator.flow_from_directory(
            train_dir,
            target_size=IMG_SIZE,
            class_mode='categorical',
            shuffle=False,
            subset='validation',)
test_gen = test_generator.flow_from_directory(
            test_dir,
            target_size=IMG_SIZE,
#             class_mode='categorical',
            shuffle=False,)

Found 1192 images belonging to 7 classes.
Found 506 images belonging to 7 classes.
Found 350 images belonging to 1 classes.


In [59]:
resnet = tf.keras.applications.DenseNet201(input_shape = (224,224,3), include_top = False)
# print(help(tf.keras.applications.ResNet50(input_shape = IMG_SHAPE, include_top = False)))

In [7]:
resnet.trainable = False

In [8]:
inputs = tf.keras.Input((224,224,3))
x = resnet(inputs)
print(x.shape)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.BatchNormalization()(x)
# x = tf.keras.layers.Dense(1024, activation='relu')(x)
# x = tf.keras.layers.Dropout(0.35)(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.25)(x)
x = tf.keras.layers.Dense(256, activation='relu')(x)
x = tf.keras.layers.Dropout(0.25)(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
outputs = tf.keras.layers.Dense(7, activation='softmax')(x)

model = tf.keras.Model(inputs,outputs)

(None, 7, 7, 1920)


In [60]:
import tensorflow_addons as tfa

callback = [tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=7, restore_best_weights=True),
            tf.keras.callbacks.ReduceLROnPlateau(factor=0.1, patience=4)]

model.compile(
#     optimizer='adam',
    optimizer=tf.keras.optimizers.Adam(0.001),
#     optimizer=tfa.optimizers.AdamW(0.001),
              loss='categorical_crossentropy',
              metrics=['acc'])

In [10]:
model.fit(train_gen,
          validation_data = val_gen,
          epochs=80,
          callbacks=callback
         )

Epoch 1/80
38/38 [==============================] - 28s 543ms/step - loss: 1.2404 - acc: 0.5369 - val_loss: 0.7975 - val_acc: 0.6937 - lr: 0.0010
Epoch 2/80
38/38 [==============================] - 11s 284ms/step - loss: 0.7148 - acc: 0.7424 - val_loss: 0.6289 - val_acc: 0.7747 - lr: 0.0010
Epoch 3/80
38/38 [==============================] - 11s 286ms/step - loss: 0.6012 - acc: 0.7894 - val_loss: 0.5732 - val_acc: 0.7984 - lr: 0.0010
Epoch 4/80
38/38 [==============================] - 11s 287ms/step - loss: 0.4954 - acc: 0.8230 - val_loss: 0.5781 - val_acc: 0.8123 - lr: 0.0010
Epoch 5/80
38/38 [==============================] - 11s 284ms/step - loss: 0.5050 - acc: 0.8205 - val_loss: 0.5620 - val_acc: 0.8043 - lr: 0.0010
Epoch 6/80
38/38 [==============================] - 11s 286ms/step - loss: 0.3926 - acc: 0.8641 - val_loss: 0.5253 - val_acc: 0.8182 - lr: 0.0010
Epoch 7/80
38/38 [==============================] - 11s 281ms/step - loss: 0.3987 - acc: 0.8641 - val_loss: 0.5700 - val_acc

In [204]:
import numpy as np
import pandas as pd
pred = model.predict(test_gen, verbose=1)
print(pred.shape)
print(pred)
answer = np.array([y.argmax() for y in pred])
print(answer)
test_df = pd.read_csv("D:\dataset\programmers/test_answer_sample_.csv")
test_df.iloc[:,1] = answer
print(test_df)
test_df.to_csv('res_net_50v2.csv', index=False)

11/11 [==============================] - 3s 164ms/step
(350, 7)
[[2.8807041e-04 4.5052871e-06 9.9960905e-01 ... 4.2948872e-07
  2.5535095e-07 9.7795993e-05]
 [4.7669730e-09 1.2208299e-09 7.6374596e-09 ... 3.9268633e-09
  1.9015747e-09 2.3959208e-08]
 [2.5468612e-02 1.8907938e-02 3.0542307e-03 ... 1.3343780e-03
  2.1876262e-04 4.1225296e-04]
 ...
 [8.5062544e-05 5.4363891e-06 1.4009899e-04 ... 7.7554942e-06
  9.7386385e-07 2.8652963e-05]
 [4.8151100e-04 8.8206250e-03 4.8535258e-05 ... 6.3392438e-04
  9.6715552e-01 1.4529819e-02]
 [1.7788501e-02 7.8034377e-01 1.1232614e-01 ... 8.3259024e-02
  5.5934809e-04 5.2302713e-03]]
[2 3 3 3 6 3 4 4 3 4 6 2 0 1 3 1 2 0 6 3 3 5 2 1 0 5 1 2 0 5 2 5 6 2 0 5 5
 4 5 1 4 0 2 3 6 3 0 5 5 3 6 5 4 1 5 0 4 5 1 1 5 0 6 1 1 2 1 1 1 3 1 3 0 1
 1 6 2 0 3 4 1 6 1 2 6 4 3 6 1 1 2 5 1 0 5 6 3 3 1 6 5 6 6 0 6 2 5 3 0 4 4
 6 2 5 4 2 0 0 5 6 4 2 2 6 4 1 5 6 0 4 1 1 6 4 4 2 5 6 5 0 4 3 1 5 1 5 4 0
 2 5 6 1 6 3 3 0 0 1 4 5 2 4 6 2 3 4 1 5 6 2 1 5 3 0 0 3 2 5 3 4 2 0 3 6

In [12]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding="same", activation="relu", input_shape=(224,224,3)))
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding="same", activation="relu"))
model.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding='valid'))
model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, padding="same", activation="relu"))
model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, padding="same", activation="relu"))
model.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding='valid'))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=128, activation='relu'))
model.add(tf.keras.layers.Dense(units=7, activation='softmax'))

model.compile(
#     optimizer='adam',
    optimizer=tf.keras.optimizers.Adam(0.001),
#     optimizer=tfa.optimizers.AdamW(0.001),
              loss='categorical_crossentropy',
              metrics=['acc'])

model.fit(train_gen,
          validation_data = val_gen,
          epochs=80,
          callbacks=callback
         )

Epoch 1/80
38/38 [==============================] - 11s 268ms/step - loss: 2.0525 - acc: 0.2081 - val_loss: 1.8680 - val_acc: 0.2352 - lr: 0.0010
Epoch 2/80
38/38 [==============================] - 10s 253ms/step - loss: 1.8496 - acc: 0.2458 - val_loss: 1.8361 - val_acc: 0.3043 - lr: 0.0010
Epoch 3/80
38/38 [==============================] - 10s 252ms/step - loss: 1.8065 - acc: 0.2945 - val_loss: 1.7982 - val_acc: 0.3142 - lr: 0.0010
Epoch 4/80
38/38 [==============================] - 10s 250ms/step - loss: 1.7985 - acc: 0.2852 - val_loss: 1.7436 - val_acc: 0.3617 - lr: 0.0010
Epoch 5/80
38/38 [==============================] - 10s 252ms/step - loss: 1.7759 - acc: 0.2919 - val_loss: 1.7478 - val_acc: 0.3676 - lr: 0.0010
Epoch 6/80
38/38 [==============================] - 10s 251ms/step - loss: 1.7655 - acc: 0.3087 - val_loss: 1.7477 - val_acc: 0.3182 - lr: 0.0010
Epoch 7/80
38/38 [==============================] - 10s 251ms/step - loss: 1.7841 - acc: 0.2961 - val_loss: 1.7326 - val_acc

In [18]:
# print(help(tfa.activations))
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding="same", activation=tfa.activations.mish, input_shape=(224,224,3)))
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding="same", activation=tfa.activations.mish))
model.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding='valid'))
model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, padding="same", activation=tfa.activations.mish))
model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, padding="same", activation=tfa.activations.mish))
model.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding='valid'))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=128, activation=tfa.activations.mish))
model.add(tf.keras.layers.Dense(units=7, activation='softmax'))

model.compile(
#     optimizer='adam',
    optimizer=tf.keras.optimizers.Adam(0.001),
#     optimizer=tfa.optimizers.AdamW(0.001),
              loss='categorical_crossentropy',
              metrics=['acc'])

model.fit(train_gen,
          validation_data = val_gen,
          epochs=80,
          callbacks=callback
         )

Epoch 1/80
38/38 [==============================] - 11s 269ms/step - loss: 2.3361 - acc: 0.2265 - val_loss: 2.0558 - val_acc: 0.2490 - lr: 0.0010
Epoch 2/80
38/38 [==============================] - 10s 260ms/step - loss: 1.8320 - acc: 0.2903 - val_loss: 1.9529 - val_acc: 0.2016 - lr: 0.0010
Epoch 3/80
38/38 [==============================] - 10s 275ms/step - loss: 1.9049 - acc: 0.2349 - val_loss: 1.7309 - val_acc: 0.3538 - lr: 0.0010
Epoch 4/80
38/38 [==============================] - 10s 274ms/step - loss: 1.8332 - acc: 0.2768 - val_loss: 1.7531 - val_acc: 0.2628 - lr: 0.0010
Epoch 5/80
38/38 [==============================] - 10s 278ms/step - loss: 1.7748 - acc: 0.3255 - val_loss: 1.7840 - val_acc: 0.2945 - lr: 0.0010
Epoch 6/80
38/38 [==============================] - 10s 276ms/step - loss: 1.7800 - acc: 0.2861 - val_loss: 1.8014 - val_acc: 0.3004 - lr: 0.0010
Epoch 7/80
38/38 [==============================] - 11s 282ms/step - loss: 1.7399 - acc: 0.3263 - val_loss: 1.7266 - val_acc

In [53]:

# print(help(tfa.activations))
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding="same", activation=tfa.activations.mish, input_shape=(224,224,3)))
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding="same", activation=tfa.activations.mish))
model.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding='valid'))
model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, padding="same", activation=tfa.activations.mish))
model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, padding="same", activation=tfa.activations.mish))
model.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding='valid'))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=128, activation=tfa.activations.mish, kernel_regularizer=tf.keras.regularizers.l2(0.001)))
model.add(tf.keras.layers.Dense(units=7, activation='softmax'))

model.compile(
#     optimizer='adam',
    optimizer=tf.keras.optimizers.Adam(0.001),
#     optimizer=tfa.optimizers.AdamW(0.001),
              loss='categorical_crossentropy',
              metrics=['acc'])

model.fit(train_gen,
          validation_data = val_gen,
          epochs=80,
          callbacks=callback
         )

ResourceExhaustedError: failed to allocate memory [Op:Mul]

In [27]:
#he초기화는 별로....

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding="same", activation='relu', input_shape=(224,224,3), kernel_initializer=tf.keras.initializers.HeNormal()))
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding="same", activation='relu'))
model.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding='valid'))
model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, padding="same", activation='relu'))
model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, padding="same", activation='relu'))
model.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding='valid'))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)))
model.add(tf.keras.layers.Dense(units=7, activation='softmax'))

model.compile(
#     optimizer='adam',
    optimizer=tf.keras.optimizers.Adam(0.001),
#     optimizer=tfa.optimizers.AdamW(0.001),
              loss='categorical_crossentropy',
              metrics=['acc'])

model.fit(train_gen,
          validation_data = val_gen,
          epochs=80,
          callbacks=callback
         )

Epoch 1/80
38/38 [==============================] - 10s 257ms/step - loss: 4.1024 - acc: 0.2047 - val_loss: 2.2590 - val_acc: 0.2213 - lr: 0.0010
Epoch 2/80
38/38 [==============================] - 10s 251ms/step - loss: 2.2153 - acc: 0.2290 - val_loss: 2.1402 - val_acc: 0.2332 - lr: 0.0010
Epoch 3/80
38/38 [==============================] - 9s 249ms/step - loss: 2.0961 - acc: 0.2273 - val_loss: 2.0504 - val_acc: 0.2273 - lr: 0.0010
Epoch 4/80
38/38 [==============================] - 10s 251ms/step - loss: 2.0319 - acc: 0.2492 - val_loss: 1.9746 - val_acc: 0.2826 - lr: 0.0010
Epoch 5/80
38/38 [==============================] - 10s 251ms/step - loss: 1.9864 - acc: 0.2609 - val_loss: 1.9367 - val_acc: 0.3004 - lr: 0.0010
Epoch 6/80
38/38 [==============================] - 10s 252ms/step - loss: 1.9365 - acc: 0.2869 - val_loss: 1.8856 - val_acc: 0.3043 - lr: 0.0010
Epoch 7/80
38/38 [==============================] - 10s 251ms/step - loss: 1.9124 - acc: 0.3003 - val_loss: 1.8515 - val_acc:

In [62]:

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(3,1), padding="same", activation=tfa.activations.mish, input_shape=(224,224,3)))
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(1,3), padding="same", activation=tfa.activations.mish))
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(3,1), padding="same", activation=tfa.activations.mish))
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(1,3), padding="same", activation=tfa.activations.mish))
model.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding='valid'))
model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, padding="same", activation=tfa.activations.mish))
model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, padding="same", activation=tfa.activations.mish))
model.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding='valid'))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=128, activation=tfa.activations.mish, kernel_regularizer=tf.keras.regularizers.l2(0.001)))
model.add(tf.keras.layers.Dense(units=7, activation='softmax'))
model.summary()
model.compile(
#     optimizer='adam',
    optimizer=tf.keras.optimizers.Adam(0.001),
#     optimizer=tfa.optimizers.AdamW(0.001),
              loss='categorical_crossentropy',
              metrics=['acc'])

model.fit(train_gen,
          validation_data = val_gen,
          epochs=80,
          callbacks=callback
         )

Model: "sequential_42"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_178 (Conv2D)         (None, 224, 224, 32)      320       
                                                                 
 conv2d_179 (Conv2D)         (None, 224, 224, 32)      3104      
                                                                 
 conv2d_180 (Conv2D)         (None, 224, 224, 32)      3104      
                                                                 
 conv2d_181 (Conv2D)         (None, 224, 224, 32)      3104      
                                                                 
 max_pooling2d_78 (MaxPoolin  (None, 112, 112, 32)     0         
 g2D)                                                            
                                                                 
 conv2d_182 (Conv2D)         (None, 112, 112, 64)      18496     
                                                     

In [66]:
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras import layers
import math

se_ratio = 4
expand_ratio = 6
width_coefficient = 1.0
depth_coefficient = 1.0
default_resolution = 112
input_channels = 3
depth_divisor= 8 
dropout_rate = 0.2
drop_connect_rate = 0.2

kernel_size = [3,3,5,3,5,5,3]
num_repeat = [1,2,2,3,3,4,1]
output_filters = [16,24,40,80,112,192,320]
# strides = [1,2,1,2,1,2,1]
strides = [1,2,2,2,1,2,1]
MBConvBlock_1_True  =  [True,False,False,False,False,False,False]

def round_repeats(repeats, depth_coefficient):
    return int(math.ceil(depth_coefficient * repeats))

def round_filters(filters, width_coefficient, depth_divisor):
    filters *= width_coefficient
    new_filters = int(filters + depth_divisor / 2) // depth_divisor * depth_divisor
    new_filters = max(depth_divisor, new_filters)
    if new_filters < 0.9 * filters:
        new_filters += depth_divisor
    return int(new_filters)

class DropConnect(layers.Layer):
    def __init__(self, drop_connect_rate=0.0, **kwargs):
        super().__init__(**kwargs)
        self.drop_connect_rate = drop_connect_rate

    def call(self, inputs, training):
        def _drop_connect():
            keep_prob = 1.0 - self.drop_connect_rate

            batch_size = tf.shape(inputs)[0]
            random_tensor = keep_prob
            random_tensor += K.random_uniform([batch_size, 1, 1, 1], dtype=inputs.dtype)
            binary_tensor = tf.floor(random_tensor)
            output = tf.math.divide(inputs, keep_prob) * binary_tensor
            return output

        return K.in_train_phase(_drop_connect, inputs, training=training)
    
def SEBlock(filters,reduced_filters):
    def _block(inputs):
        x = layers.GlobalAveragePooling2D()(inputs)
        x = layers.Reshape((1,1,x.shape[1]))(x)
        x = layers.Conv2D(reduced_filters, 1, 1)(x)
        x = tfa.activations.mish(x)
        x = layers.Conv2D(filters, 1, 1)(x)
        x = layers.Activation('sigmoid')(x)
        x = layers.Multiply()([x, inputs])
        return x
    return _block

def MBConvBlock(x,kernel_size, strides,drop_connect_rate,output_channels,MBConvBlock_1_True=False):
    output_channels = round_filters(output_channels,width_coefficient,depth_divisor)
    if MBConvBlock_1_True:
        block = layers.DepthwiseConv2D(kernel_size, strides,padding='same', use_bias=False)(x)
        block = layers.BatchNormalization()(block)
        block = tfa.activations.mish(block)
        block = SEBlock(x.shape[3],x.shape[3]/se_ratio)(block)
        block = layers.Conv2D(output_channels, (1,1), padding='same', use_bias=False)(block)
        block = layers.BatchNormalization()(block)
        return block

    channels = x.shape[3]
    expand_channels = channels * expand_ratio
    block = layers.Conv2D(expand_channels, (1,1), padding='same', use_bias=False)(x)
    block = layers.BatchNormalization()(block)
    block = tfa.activations.mish(block)
    block = layers.DepthwiseConv2D(kernel_size, strides,padding='same', use_bias=False)(block)
    block = layers.BatchNormalization()(block)
    block = tfa.activations.mish(block)
    block = SEBlock(expand_channels,channels/se_ratio)(block)
    block = layers.Conv2D(output_channels, (1,1), padding='same', use_bias=False)(block)
    block = layers.BatchNormalization()(block)
    if x.shape[3] == output_channels:
        block = DropConnect(drop_connect_rate)(block)
        block = layers.Add()([block, x])
    return block

def Build_Network(num_classes, booster=False):
    x_input = layers.Input(shape=(default_resolution,default_resolution,input_channels))    
    x = layers.Conv2D(round_filters(32, width_coefficient, depth_divisor), (3,3), 2,padding='same', use_bias=False)(x_input)
    x = layers.BatchNormalization()(x)
    x = tfa.activations.mish(x)
    num_blocks_total = sum(num_repeat)
    block_num = 0
    for i in range(len(kernel_size)):
        round_num_repeat = round_repeats(num_repeat[i], depth_coefficient)
        drop_rate = drop_connect_rate * float(block_num) / num_blocks_total
        x = MBConvBlock(x,kernel_size[i],strides[i],drop_rate,output_filters[i],MBConvBlock_1_True = MBConvBlock_1_True[i])
        block_num += 1
        if round_num_repeat > 1:
            for bidx in range(round_num_repeat - 1):
                drop_rate = drop_connect_rate * float(block_num) / num_blocks_total
                x = MBConvBlock(x,kernel_size[i],1,drop_rate,output_filters[i],MBConvBlock_1_True = MBConvBlock_1_True[i])
                block_num += 1
    x = layers.Conv2D(round_filters(1280, width_coefficient, depth_divisor), 1,padding='same',use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = tfa.activations.mish(x)
    if booster==False:
        x = layers.GlobalAveragePooling2D()(x)
        x = layers.Dropout(dropout_rate)(x)
        x = layers.Dense(num_classes,activation='softmax')(x)
    model = tf.keras.models.Model(inputs=x_input, outputs=x)
    return model

model = Build_Network(7)

model.compile(
#     optimizer='adam',
    optimizer=tf.keras.optimizers.Adam(0.001),
#     optimizer=tfa.optimizers.AdamW(0.001),
              loss='categorical_crossentropy',
              metrics=['acc'])

model.fit(train_gen,
          validation_data = val_gen,
          epochs=80,
          callbacks=callback
         )

Epoch 1/80
38/38 [==============================] - 16s 299ms/step - loss: 2.0971 - acc: 0.2517 - val_loss: 1.9155 - val_acc: 0.2352 - lr: 0.0010
Epoch 2/80
38/38 [==============================] - 10s 258ms/step - loss: 1.8606 - acc: 0.2802 - val_loss: 1.9032 - val_acc: 0.2352 - lr: 0.0010
Epoch 3/80
38/38 [==============================] - 10s 260ms/step - loss: 1.8362 - acc: 0.3070 - val_loss: 1.8942 - val_acc: 0.2352 - lr: 0.0010
Epoch 4/80
38/38 [==============================] - 10s 260ms/step - loss: 1.8248 - acc: 0.2903 - val_loss: 1.8912 - val_acc: 0.1937 - lr: 0.0010
Epoch 5/80
38/38 [==============================] - 10s 260ms/step - loss: 1.7883 - acc: 0.2894 - val_loss: 1.8835 - val_acc: 0.2352 - lr: 0.0010
Epoch 6/80
38/38 [==============================] - 10s 264ms/step - loss: 1.7491 - acc: 0.3138 - val_loss: 1.8849 - val_acc: 0.2352 - lr: 0.0010
Epoch 7/80
38/38 [==============================] - ETA: 0s - loss: 1.7301 - acc: 0.3297

KeyboardInterrupt: 

In [75]:
def SEBlock(input_feature, ratio=16):
    """Squeeze and Excitation Block."""
    channel_axis = -1
    channel = input_feature.shape[channel_axis]
    
    se_shape = (1, 1, channel)
    se = GlobalAveragePooling2D()(input_feature)
    se = Dense(channel // ratio, activation='relu', use_bias=False)(se)
    se = Dense(channel, activation='sigmoid', use_bias=False)(se)
    se = Reshape(se_shape)(se)
    return Multiply()([input_feature, se])

def conv_and_se_block(x, filters):
    """A helper function to apply Convolution + SE Block."""
    x = Conv2D(filters, (3, 3), padding="same", use_bias=False)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = SEBlock(x)
    return x

def depthwise_sep_conv_and_se_block(x, filters):
    """A helper function to apply Depthwise Separable Conv + SE Block."""
    x = DepthwiseConv2D((3, 3), padding="same", use_bias=False)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = Conv2D(filters, (1, 1), padding="same", use_bias=False)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = SEBlock(x)
    return x

def make_model(input_shape, num_classes):
    inputs = Input(shape=input_shape)

    x = conv_and_se_block(inputs, 32)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    
    x = depthwise_sep_conv_and_se_block(x, 64)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    
    x = depthwise_sep_conv_and_se_block(x, 128)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    # Additional depth for larger input size
    x = depthwise_sep_conv_and_se_block(x, 256)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    x = depthwise_sep_conv_and_se_block(x, 512)
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)
    
    outputs = Dense(num_classes, activation="softmax")(x)

    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
    return model

# 모델 생성 (예: CIFAR-10의 입력 크기와 클래스 수 사용)
model = make_model((224, 224, 3), 7)
model.summary()

model.compile(
    optimizer='adam',
#     optimizer=tf.keras.optimizers.Adam(0.001),
#     optimizer=tfa.optimizers.AdamW(0.001),
              loss='categorical_crossentropy',
              metrics=['acc'])

model.fit(train_gen,
          validation_data = val_gen,
          epochs=80,
          callbacks=callback
         )

Model: "model_10"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_15 (InputLayer)          [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_296 (Conv2D)            (None, 224, 224, 32  864         ['input_15[0][0]']               
                                )                                                                 
                                                                                                  
 batch_normalization_130 (Batch  (None, 224, 224, 32  128        ['conv2d_296[0][0]']             
 Normalization)                 )                                                          

38/38 [==============================] - 11s 269ms/step - loss: 1.8064 - acc: 0.3012 - val_loss: 1.9126 - val_acc: 0.2352 - lr: 0.0010
Epoch 2/80
38/38 [==============================] - 10s 255ms/step - loss: 1.6863 - acc: 0.3616 - val_loss: 1.8950 - val_acc: 0.2352 - lr: 0.0010
Epoch 3/80
38/38 [==============================] - 10s 257ms/step - loss: 1.6509 - acc: 0.3473 - val_loss: 1.9041 - val_acc: 0.2352 - lr: 0.0010
Epoch 4/80
13/38 [=========>....................] - ETA: 4s - loss: 1.5281 - acc: 0.4362

KeyboardInterrupt: 

In [97]:
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras import layers, models, regularizers

# def SEBlock(input_feature, ratio=16):
#     channel_axis = -1
#     channel = input_feature.shape[channel_axis]
    
#     se_shape = (1, 1, channel)
#     se = layers.GlobalAveragePooling2D()(input_feature)
#     se = layers.Reshape(se_shape)(se)
#     se = layers.Dense(channel // ratio, activation='relu', use_bias=False)(se)
#     se = layers.Dense(channel, activation='sigmoid', use_bias=False)(se)
#     se = layers.Reshape(se_shape)(se)
#     return layers.Multiply()([input_feature, se])

# def conv_and_se_block(x, filters):
#     x = layers.Conv2D(filters, 3, padding="same")(x)
#     x = tfa.activations.mish(x)
#     x = layers.DepthwiseConv2D(3, padding="same")(x)
#     x = tfa.activations.mish(x)
#     x = layers.Conv2D(filters, 1, padding="same")(x)
#     x = tfa.activations.mish(x)
#     x = SEBlock(x)
#     return x

# def make_model(input_shape=(224, 224, 3), num_classes=7):
#     inputs = tf.keras.Input(shape=input_shape)
    
#     x = conv_and_se_block(inputs, 32)
#     x = layers.MaxPooling2D(pool_size=2, strides=2)(x)
    
#     x = conv_and_se_block(x, 64)
#     x = layers.MaxPooling2D(pool_size=2, strides=2)(x)
    
# #     x = conv_and_se_block(x, 128)
# #     x = layers.MaxPooling2D(pool_size=2, strides=2)(x)

#     x = layers.Flatten()(x)
#     x = layers.Dense(128, activation=tfa.activations.mish, kernel_regularizer=regularizers.l2(0.001))(x)
#     x = layers.Dense(num_classes, activation='softmax')(x)

#     model = models.Model(inputs=inputs, outputs=x)
#     model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
#     return model

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(3,1), padding="same", activation=tfa.activations.mish, input_shape=(224,224,3)))
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(1,3), padding="same", activation=tfa.activations.mish))
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(3,1), padding="same", activation=tfa.activations.mish))
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(1,3), padding="same", activation=tfa.activations.mish))
model.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding='valid'))
model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, padding="same", activation=tfa.activations.mish))
model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, padding="same", activation=tfa.activations.mish))
model.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding='valid'))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=128, activation=tfa.activations.mish, kernel_regularizer=tf.keras.regularizers.l2(0.001)))
model.add(tf.keras.layers.Dense(units=7, activation='softmax'))
model.summary()

# 모델 생성
# model = make_model()

# focal_loss = tfa.losses.SigmoidFocalCrossEntropy()
def categorical_focal_loss(gamma=2.0, alpha=0.25):
    """
    Focal Loss for multi-class or categorical classification.
    This loss function generalizes the idea of Focal Loss to multi-class classification.
    """
    def focal_loss_fixed(y_true, y_pred):
        """
        Focal loss calculation.
        """
        # Clip the prediction value to prevent NaN's and Inf's
        epsilon = tf.keras.backend.epsilon()
        y_pred = tf.clip_by_value(y_pred, epsilon, 1. - epsilon)

        # Calculate Cross Entropy
        cross_entropy = -y_true * tf.math.log(y_pred)

        # Calculate Focal Loss
        loss = alpha * tf.pow(1 - y_pred, gamma) * cross_entropy

        # Sum the losses in mini_batch
        return tf.reduce_sum(loss, axis=1)
    
    return focal_loss_fixed


model.compile(
#     optimizer='adam',
    optimizer=tf.keras.optimizers.Adam(0.001),
#     optimizer=tfa.optimizers.AdamW(0.001),
              loss=categorical_focal_loss(gamma=2., alpha=0.25),
#               loss='categorical_crossentropy',
              metrics=['acc'])

model.fit(train_gen,
          validation_data = val_gen,
          epochs=80,
          callbacks=callback
         )

Model: "sequential_46"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_385 (Conv2D)         (None, 224, 224, 32)      320       
                                                                 
 conv2d_386 (Conv2D)         (None, 224, 224, 32)      3104      
                                                                 
 conv2d_387 (Conv2D)         (None, 224, 224, 32)      3104      
                                                                 
 conv2d_388 (Conv2D)         (None, 224, 224, 32)      3104      
                                                                 
 max_pooling2d_149 (MaxPooli  (None, 112, 112, 32)     0         
 ng2D)                                                           
                                                                 
 conv2d_389 (Conv2D)         (None, 112, 112, 64)      18496     
                                                     

In [102]:
base_model = tf.keras.applications.EfficientNetB0(include_top=False,
                                                  weights='imagenet',
                                                  input_shape=(224, 224, 3))

# 기본 모델의 층을 고정 (가중치를 업데이트하지 않음)
base_model.trainable = False

# 모델 구축
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),  # 특징 맵을 하나의 특징 벡터로 변환
    layers.Dense(1024, activation='relu'),  # 새로운 분류 층
    layers.Dropout(0.2),
    layers.Dense(7, activation='softmax')  # 가정: 10개의 클래스가 있다
])

model.compile(
    optimizer='rmsprop',
#     optimizer=tf.keras.optimizers.Adam(0.001),
#     optimizer=tfa.optimizers.AdamW(0.001),
              loss=categorical_focal_loss(gamma=2., alpha=0.25),
#               loss='categorical_crossentropy',
              metrics=['acc'])

model.fit(train_gen,
          validation_data = val_gen,
          epochs=80,
          callbacks=callback
         )

Epoch 1/80
38/38 [==============================] - 13s 277ms/step - loss: 0.3657 - acc: 0.2097 - val_loss: 0.3417 - val_acc: 0.2352 - lr: 0.0010
Epoch 2/80
38/38 [==============================] - 10s 257ms/step - loss: 0.3480 - acc: 0.2257 - val_loss: 0.3455 - val_acc: 0.2352 - lr: 0.0010
Epoch 3/80
38/38 [==============================] - 10s 258ms/step - loss: 0.3471 - acc: 0.2089 - val_loss: 0.3464 - val_acc: 0.2352 - lr: 0.0010
Epoch 4/80
38/38 [==============================] - 10s 260ms/step - loss: 0.3472 - acc: 0.2206 - val_loss: 0.3421 - val_acc: 0.2352 - lr: 0.0010
Epoch 5/80
38/38 [==============================] - 10s 258ms/step - loss: 0.3439 - acc: 0.2232 - val_loss: 0.3448 - val_acc: 0.2352 - lr: 0.0010
Epoch 6/80
38/38 [==============================] - 10s 263ms/step - loss: 0.3433 - acc: 0.2315 - val_loss: 0.3404 - val_acc: 0.2352 - lr: 1.0000e-04
Epoch 7/80
38/38 [==============================] - 10s 259ms/step - loss: 0.3419 - acc: 0.2315 - val_loss: 0.3399 - val

In [108]:
import tensorflow as tf
import os
import matplotlib.pyplot as plt
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow_addons as tfa
import numpy as np
import pandas as pd

base_dir = 'D:/dataset/programmers/'
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')
train_dir, test_dir

IMG_SIZE = (224,224)
generator = ImageDataGenerator(rotation_range=40,
                          width_shift_range=0.2,
                          height_shift_range=0.2,
#                           shear_range=0.2,
#                           zoom_range=0.2,
                          horizontal_flip=True,
#                           vertical_flip=True,
                          fill_mode = 'reflect',
                          validation_split = 0.3,
                          rescale=1/255.)

test_generator = ImageDataGenerator(rescale=1/255.)
# print(help(ImageDataGenerator))

train_gen = generator.flow_from_directory(
            train_dir,
            target_size=IMG_SIZE,
            class_mode='categorical',
            shuffle=True,
            subset='training',)
val_gen = generator.flow_from_directory(
            train_dir,
            target_size=IMG_SIZE,
            class_mode='categorical',
            shuffle=False,
            subset='validation',)
test_gen = test_generator.flow_from_directory(
            test_dir,
            target_size=IMG_SIZE,
#             class_mode='categorical',
            shuffle=False,)

base_model = tf.keras.applications.EfficientNetB0(include_top=False,
                                                  weights='imagenet',
                                                  input_shape=(224, 224, 3))

# 기본 모델의 층을 고정 (가중치를 업데이트하지 않음)
base_model.trainable = False

# 모델 구축
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),  # 특징 맵을 하나의 특징 벡터로 변환
    layers.Dense(512, activation='relu'),  # 새로운 분류 층
#     layers.Dropout(0.2),
    layers.Dense(7, activation='softmax')  # 가정: 10개의 클래스가 있다
])
model.summary()
def categorical_focal_loss(gamma=2.0, alpha=0.25):
    """
    Focal Loss for multi-class or categorical classification.
    This loss function generalizes the idea of Focal Loss to multi-class classification.
    """
    def focal_loss_fixed(y_true, y_pred):
        """
        Focal loss calculation.
        """
        # Clip the prediction value to prevent NaN's and Inf's
        epsilon = tf.keras.backend.epsilon()
        y_pred = tf.clip_by_value(y_pred, epsilon, 1. - epsilon)

        # Calculate Cross Entropy
        cross_entropy = -y_true * tf.math.log(y_pred)

        # Calculate Focal Loss
        loss = alpha * tf.pow(1 - y_pred, gamma) * cross_entropy

        # Sum the losses in mini_batch
        return tf.reduce_sum(loss, axis=1)
    
    return focal_loss_fixed

callback = [tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=7, restore_best_weights=True),
            tf.keras.callbacks.ReduceLROnPlateau(factor=0.1, patience=4)]

model.compile(
    optimizer='adam',
#     optimizer=tf.keras.optimizers.Adam(0.001),
#     optimizer=tfa.optimizers.AdamW(0.001),
              loss=categorical_focal_loss(gamma=2., alpha=0.25),
#               loss='categorical_crossentropy',
              metrics=['acc'])

model.fit(train_gen,
          validation_data = val_gen,
          epochs=80,
          callbacks=callback
         )

Found 1192 images belonging to 7 classes.
Found 506 images belonging to 7 classes.
Found 350 images belonging to 1 classes.
Model: "sequential_57"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb0 (Functional)  (None, 7, 7, 1280)       4049571   
                                                                 
 global_average_pooling2d_10  (None, 1280)             0         
 5 (GlobalAveragePooling2D)                                      
                                                                 
 dense_267 (Dense)           (None, 512)               655872    
                                                                 
 dense_268 (Dense)           (None, 7)                 3591      
                                                                 
Total params: 4,709,034
Trainable params: 659,463
Non-trainable params: 4,049,571
_____________________________________________

In [9]:
import tensorflow as tf
import os
import matplotlib.pyplot as plt
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow_addons as tfa
import numpy as np
import pandas as pd
from tensorflow.keras import layers
from tensorflow.keras import models
base_dir = 'D:/dataset/cifar10/'
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')
train_dir, test_dir

IMG_SIZE = (224,224)

train_ds = tf.keras.utils.image_dataset_from_directory(
  train_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(224, 224),
  batch_size=32)
val_ds = tf.keras.utils.image_dataset_from_directory(
  train_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(224, 224),
  batch_size=32)
test_ds = tf.keras.utils.image_dataset_from_directory(
  test_dir,
  seed=123,
  image_size=(224, 224),
  batch_size=32)
# import tensorflow_datasets as tfds

data_augmentation = tf.keras.Sequential([
    layers.Rescaling(1. / 255),
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2),
])

base_model = tf.keras.applications.ResNet50(include_top=False,
                                                  weights='imagenet',
                                                  input_shape=(224, 224, 3))

# 기본 모델의 층을 고정 (가중치를 업데이트하지 않음)
base_model.trainable = False

# 모델 구축
model = models.Sequential([
    data_augmentation,
    base_model,
    layers.GlobalAveragePooling2D(),  # 특징 맵을 하나의 특징 벡터로 변환
    layers.Dense(512, activation='relu'),  # 새로운 분류 층
#     layers.Dropout(0.2),
    layers.Dense(10, activation='softmax')  # 가정: 10개의 클래스가 있다
])
# model.summary()
def categorical_focal_loss(gamma=2.0, alpha=0.25):
    """
    Focal Loss for multi-class or categorical classification.
    This loss function generalizes the idea of Focal Loss to multi-class classification.
    """
    def focal_loss_fixed(y_true, y_pred):
        """
        Focal loss calculation.
        """
        # Clip the prediction value to prevent NaN's and Inf's
        epsilon = tf.keras.backend.epsilon()
        y_pred = tf.clip_by_value(y_pred, epsilon, 1. - epsilon)

        # Calculate Cross Entropy
        cross_entropy = -y_true * tf.math.log(y_pred)

        # Calculate Focal Loss
        loss = alpha * tf.pow(1 - y_pred, gamma) * cross_entropy

        # Sum the losses in mini_batch
        return tf.reduce_sum(loss, axis=1)
    
    return focal_loss_fixed

callback = [tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=7, restore_best_weights=True),
            tf.keras.callbacks.ReduceLROnPlateau(factor=0.1, patience=4)]

model.compile(
    optimizer='adam',
#     optimizer=tf.keras.optimizers.Adam(0.001),
#     optimizer=tfa.optimizers.AdamW(0.001),
#               loss=categorical_focal_loss(gamma=2., alpha=0.25),
              loss='sparse_categorical_crossentropy',
              metrics=['acc'])

model.fit(train_ds,
          validation_data = val_ds,
          epochs=80,
          callbacks=callback
         )

Found 50000 files belonging to 10 classes.
Using 40000 files for training.
Found 50000 files belonging to 10 classes.
Using 10000 files for validation.
Found 10000 files belonging to 10 classes.
Epoch 1/80
1250/1250 [==============================] - 104s 81ms/step - loss: 2.2930 - acc: 0.1253 - val_loss: 2.2479 - val_acc: 0.1521 - lr: 0.0010
Epoch 2/80
1250/1250 [==============================] - 101s 81ms/step - loss: 2.2365 - acc: 0.1498 - val_loss: 2.2318 - val_acc: 0.1526 - lr: 0.0010
Epoch 3/80
1250/1250 [==============================] - 102s 81ms/step - loss: 2.2122 - acc: 0.1633 - val_loss: 2.1934 - val_acc: 0.1817 - lr: 0.0010
Epoch 4/80
1250/1250 [==============================] - 101s 81ms/step - loss: 2.1877 - acc: 0.1760 - val_loss: 2.1657 - val_acc: 0.1910 - lr: 0.0010
Epoch 5/80
1250/1250 [==============================] - 101s 81ms/step - loss: 2.1608 - acc: 0.1889 - val_loss: 2.1283 - val_acc: 0.2168 - lr: 0.0010
Epoch 6/80
1250/1250 [==============================] -

KeyboardInterrupt: 

In [7]:
help(tf.keras.applications)

Help on package keras.api._v2.keras.applications in keras.api._v2.keras:

NAME
    keras.api._v2.keras.applications - Keras Applications are premade architectures with pre-trained weights.

PACKAGE CONTENTS
    convnext (package)
    densenet (package)
    efficientnet (package)
    efficientnet_v2 (package)
    imagenet_utils (package)
    inception_resnet_v2 (package)
    inception_v3 (package)
    mobilenet (package)
    mobilenet_v2 (package)
    mobilenet_v3 (package)
    nasnet (package)
    regnet (package)
    resnet (package)
    resnet50 (package)
    resnet_rs (package)
    resnet_v2 (package)
    vgg16 (package)
    vgg19 (package)
    xception (package)

FILE
    c:\users\jiwoon\anaconda3\lib\site-packages\keras\api\_v2\keras\applications\__init__.py


